In [57]:
import pandas as pd


In [58]:
world_data = pd.read_csv("data/world_bank/combined_world_data.csv")
eurovision_data = pd.read_csv("data/eurovision/cleansed_eurovision_data.csv")

In [59]:
world_data.sample(10)

,index,Country Name,Country Code,Year,Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",...,"People with basic handwashing facilities including soap and water, urban (% of urban population)","Risk premium on lending (lending rate minus treasury bill rate, %)","Incidence of malaria (per 1,000 population at risk)","Net financial flows, RDB concessional (NFL, current US$)",Financial intermediary services indirectly Measured (FISIM) (constant LCU),"Net financial flows, IMF concessional (NFL, current US$)",Newborns protected against tetanus (%),"Net official flows from UN agencies, UNWTO (current US$)",Children with fever receiving antimalarial drugs (% of children under age 5 with fever),Use of insecticide-treated bed nets (% of under-5 population)
1514,42,Germany,DEU,2002,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,44,Norway,NOR,2004,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
460,12,France,FRA,1972,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2834,18,Greece,GRC,1978,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1211,59,Serbia,SRB,2019,78.7,62.6,91.1,99.8,99.7,99.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446,38,Latvia,LVA,1998,NaN,NaN,NaN,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2958,14,Czechia,CZE,1974,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,30,Poland,POL,1990,NaN,NaN,NaN,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,53,Georgia,GEO,2013,74.2,49.0,94.9,100.0,100.0,100.0,...,NaN,8.434642,0.0,47149976.0,NaN,NaN,NaN,NaN,NaN,NaN
815,47,Belgium,BEL,2007,100.0,100.0,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
eurovision_data.sample(10)

,Song,Artist,Country,Year,Language,Grand Final Place,Grand Final Points,Semifinal,Semifinal Place,Semifinal Points
980,Celebrate,Piero Esteriore & The MusicStars,Switzerland,2004.0,English,NaN,0.0,NaN,22.0,0.0
752,Mana mou (Μάνα μου),Hara & Andreas Konstantinou,Cyprus,1997.0,Greek,5.0,98.0,NaN,NaN,NaN
1051,I Do,Fabrizio Faniello,Malta,2006.0,English,24.0,1.0,NaN,NaN,NaN
744,"¡Ay, qué deseo!",Antonio Carbonell,Spain,1996.0,Spanish,20.0,17.0,NaN,NaN,NaN
1832,Die ganze Welt braucht Liebe,Liane Augustin,Austria,1958.0,German,5.0,8.0,0.0,0.0,0.0
228,Tvoj dječak je tužan,Krunoslav Slabinac,Yugoslavia,1971.0,Serbo-Croatian,14.0,68.0,NaN,NaN,NaN
250,Junger Tag,Gitte,Germany,1973.0,German,8.0,85.0,NaN,NaN,NaN
1371,Party for Everybody,Buranovskiye Babushki,Russia,2012.0,English,2.0,259.0,1.0,1.0,152.0
712,Új név a régi ház falán,Csaba Szigeti,Hungary,1995.0,Hungarian,22.0,3.0,NaN,NaN,NaN
1276,Narodnozabavni rock,Ansambel Žlindra and Kalamari,Slovenia,2010.0,Slovene,NaN,0.0,2.0,16.0,6.0


In [61]:
eurovision_countries = eurovision_data["Country"].unique()
world_countries = world_data["Country Name"].unique()

In [62]:
for country in eurovision_countries:
    if country not in world_data["Country Name"].unique():
        print(country)

nan
Yugoslavia
Turkey
Russia
Slovakia
Serbia and Montenegro
Czech Republic


In [63]:
for country in world_countries:
    if country not in eurovision_countries:
        print(country)

Russian Federation
Slovak Republic
Czechia
Turkiye


In [64]:
eurovision_data["Country"].replace({"Czech Republic": "Czechia", "Russia": "Russian Federation", "Slovakia": "Slovak Republic", "Turkey": "Turkiye"}, inplace=True)

/var/folders/kl/zs51p_z135905dzchw5f3n480000gp/T/ipykernel_23357/796732920.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  eurovision_data["Country"].replace({"Czech Republic": "Czechia", "Russia": "Russian Federation", "Slovakia": "Slovak Republic", "Turkey": "Turkiye"}, inplace=True)


In [65]:
eurovision_data.rename(columns={"Country": "Country Name"}, inplace=True)

In [66]:
merged_df = pd.merge(eurovision_data, world_data, on=['Country Name', 'Year'], how='inner').reset_index()


In [67]:
first_cols = ['Country Name', 'Country Code', 'Year', "Song", "Artist", "Language", "Grand Final Place", "Grand Final Points", "Semifinal", "Semifinal Place", "Semifinal Points"]
remaining_cols = [col for col in merged_df.columns if col not in first_cols]
new_col_order = first_cols + remaining_cols

In [68]:
merged_df = merged_df[new_col_order]

In [69]:
merged_df.tail(10)

,Country Name,Country Code,Year,Song,Artist,Language,Grand Final Place,Grand Final Points,Semifinal,Semifinal Place,...,"People with basic handwashing facilities including soap and water, urban (% of urban population)","Risk premium on lending (lending rate minus treasury bill rate, %)","Incidence of malaria (per 1,000 population at risk)","Net financial flows, RDB concessional (NFL, current US$)",Financial intermediary services indirectly Measured (FISIM) (constant LCU),"Net financial flows, IMF concessional (NFL, current US$)",Newborns protected against tetanus (%),"Net official flows from UN agencies, UNWTO (current US$)",Children with fever receiving antimalarial drugs (% of children under age 5 with fever),Use of insecticide-treated bed nets (% of under-5 population)
3504,San Marino,SMR,2023.0,Like an Animal,Piqued Jacks,English,0.0,0.0,2.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3505,Serbia,SRB,2023.0,Samo mi se spava,Luke Black,Serbian,24.0,30.0,1.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3506,Serbia,SRB,2023.0,Samo mi se spava,Luke Black,English,24.0,30.0,1.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3507,Slovenia,SVN,2023.0,Carpe Diem,Joker Out,Slovene,21.0,78.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3508,Spain,ESP,2023.0,Eaea,Blanca Paloma,Spanish,17.0,100.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3509,Sweden,SWE,2023.0,Tattoo,Loreen,English,1.0,583.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3510,Switzerland,CHE,2023.0,Watergun,Remo Forrer,English,20.0,92.0,1.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3511,Ukraine,UKR,2023.0,Heart of Steel,Tvorchi,English,6.0,243.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3512,Ukraine,UKR,2023.0,Heart of Steel,Tvorchi,Ukrainian,6.0,243.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3513,United Kingdom,GBR,2023.0,I Wrote a Song,Mae Muller,English,25.0,24.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3514 entries, 0 to 3513
Columns: 1498 entries, Country Name to Use of insecticide-treated bed nets (% of under-5 population)
dtypes: float64(1491), int64(2), object(5)
memory usage: 40.2+ MB


In [71]:
merged_df.to_csv("data/combined_data.csv")